# Loading Model


Install required Libraries

In [1]:
!pip install -q transformers einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.9 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-wce28ccq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-wce28ccq
  Resolved https://github.com/huggingface/accelerate to commit 739b135f8367becb67ffaada12fe76e3aa60fefd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.23.0.dev0-py3-none-any.whl size=255974 sha256=68cb3c640b61d4b8c3383b4bc6afff1172054218ed396661de0d3bf

Load the libraries and Load the Tokenizer

In [2]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


model_name = "tiiuae/falcon-rw-1b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Build the Model Pipeline using Hugging Face Transformers Pipeline

In [3]:
from peft import LoraConfig

lora_alpha = 32
lora_dropout = 0.05
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)


# Dataset


In [4]:
from datasets import load_dataset

dataset_name = "pubmed_qa"
dataset = load_dataset(dataset_name, "pqa_artificial")

dataset_reformat = dataset['train'].train_test_split(test_size=9000, train_size=21000)
print(dataset_reformat)

def combine_context(row):
  row['context'] = ''.join(row['context']['contexts'])
  return row

def get_text(row):
  row['text'] = "### Context: " + row['context'] +  "### Question: " + row['question'] + "### Answer: " + row['long_answer']
  return row

dataset_train = dataset_reformat['train']
dataset_train.add_column(name="text", column=dataset_train['long_answer'])
dataset_train = dataset_train.map(combine_context)
dataset_train = dataset_train.map(get_text,  remove_columns=dataset_reformat["train"].column_names)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 21000
    })
    test: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 9000
    })
})


Flattening the indices:   0%|          | 0/21000 [00:00<?, ? examples/s]

Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 211269
    })
})


# Loading the trainer

In [6]:
from transformers import TrainingArguments



training_arguments = TrainingArguments(

    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs = 1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    max_steps = 500,
    optim = "paged_adamw_32bit",
    output_dir = "./results",
    lr_scheduler_type = "constant",
    warmup_ratio = 0.03,
    max_grad_norm = 0.3,
    save_steps = 10,
    group_by_length=True,
    report_to=None
)



Pass everything to Trainer

In [7]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

pre-process the model by upcasting the layer norms in float 32 for more stable training

In [8]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)



# Train The Model

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,1.917400
10,1.879900
15,1.920500
20,1.871000
25,1.930600
30,1.934500
35,1.947600
40,2.016000
45,2.003300
50,2.052700
